In [16]:
import pandas as pd
from konlpy.tag import Komoran, Kkma, Okt
import re
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from math import log
from operator import itemgetter

In [17]:
dataset = pd.read_csv('D:/LikeLion/Code/Project2/Data/spell_check_label.csv',encoding='utf-8')

In [18]:
with open('D:/LikeLion/Code/Project2/Data/kor_stop.txt', 'r', encoding='utf-8') as f:
    list_file = f.readlines()
stopwords = [line.rstrip('\n') for line in list_file] 

In [19]:
hotel_name = list(dataset.groupby('hotelName').count().index)
groups = dataset.groupby(dataset.hotelName)

In [20]:
for i, name in enumerate(hotel_name):
    globals()['df_{}'.format(i)] = groups.get_group(name)

In [21]:
df_0_neg = df_0[df_0['label']==0].copy()

In [22]:
df_0_neg['fixed'] = df_0_neg['fixed'].apply(lambda x: re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","",x))

In [23]:
document_count = len(df_0_neg)

In [24]:
komoran = Komoran(userdic='D:/LikeLion/Code/Project2/Data/userdic.txt')
kkma = Kkma()
okt = Okt()

In [25]:
def tokenizer(text):
    tokens_ko = komoran.morphs(text)
    to_return = []
    for words in tokens_ko:
        if words not in stopwords:
            to_return.append(words)
    return re.sub('\.','',' '.join(to_return))

In [26]:
df_0_neg['fixed'] = df_0_neg['fixed'].apply(lambda x: tokenizer(x))

In [27]:
documents = df_0_neg['fixed'].to_list()

In [28]:
def get_term_frequency(document, word_dict=None):
    if word_dict is None:
        word_dict = {}
    words = document.split()

    for w in words:
        word_dict[w] = 1 + (0 if word_dict.get(w) is None else word_dict[w])

    return pd.Series(word_dict, dtype='float64').sort_values(ascending=False)

In [29]:
def get_document_frequency(documents):
    dicts = []
    vocab = set([])
    df = {}

    for d in documents:
        tf = get_term_frequency(d)
        dicts += [tf]
        vocab = vocab | set(tf.keys())

    for v in list(vocab):
        df[v] = 0
        for dict_d in dicts:
            if dict_d.get(v) is not None:
                df[v] += 1
    
    return pd.Series(df, name='df', dtype='float64').sort_values(ascending=False)

In [30]:
def get_tf(docs):
    vocab = {}
    tfs = []
    for d in docs:
        vocab = get_term_frequency(d, vocab)
        tfs += [get_term_frequency(d)]

    stats = []

    for word, freq in vocab.items():
        tf_v = []
        for idx in range(len(docs)):
            if tfs[idx].get(word) is not None:
                tf_v += [tfs[idx][word]]
            
            else:
                tf_v += [0]

        stats.append((word, freq, *tf_v))

    column_name = ['word','totalFrequency']

    for i in range(1, len(docs)+1):
        column_name.append('document'+str(i))

    return pd.DataFrame(stats, columns=column_name).sort_values('totalFrequency', ascending=False)

In [31]:
# def get_tfidf(docs):
#     vocab = {}
#     tfs = []
#     for d in docs:
#         vocab = get_term_frequency(d, vocab)
#         tfs += [get_term_frequency(d)]
    
#     df = get_document_frequency(docs)

#     stats = []

#     for word, freq in vocab.items():
#         tfidfs = []
#         for idx in range(len(docs)):
#             if tfs[idx].get(word) is not None:
#                 tfidfs += [tfs[idx][word] * np.log(len(docs) / df[word])]
            
#             else:
#                 tfidfs += [0]

#         stats.append((word, freq, *tfidfs, max(tfidfs)))
    
#     column_name = ['word','totalFrequency']

#     for i in range(1, len(docs)+1):
#         column_name.append('document'+str(i))
    
#     column_name.append('max')

#     return pd.DataFrame(stats, columns=column_name).sort_values('max', ascending=False)

In [32]:
tf_matrix = get_tf(documents)
df_matrix = get_document_frequency(documents)

In [33]:
df_matrix

어요      98.0
ㄴ       86.0
있       74.0
안       67.0
좋       64.0
        ... 
와도       1.0
전기       1.0
개미       1.0
끈        1.0
초코파이     1.0
Name: df, Length: 1074, dtype: float64

In [34]:
tf_matrix

,word,totalFrequency,document1,document2,document3,document4,document5,document6,document7,document8,...,document415,document416,document417,document418,document419,document420,document421,document422,document423,document424
0,어요,98.0,0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,ㄴ,97.0,0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0
2,있,91.0,0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,안,72.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,좋,65.0,0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,빠지,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
681,서약서,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
682,찬물,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
683,트윈침대,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [35]:
def get_ntf(matrix):
    max_btf = max(matrix['totalFrequency'])
    total_btf = sum(matrix['totalFrequency'])

    col_names = list(matrix.columns)[2:]

    matrix['ntf1'] = matrix['totalFrequency'].apply(lambda x: x/max_btf)

    matrix_ntf2 = matrix[col_names].copy()

    matrix_ntf2 = matrix_ntf2.apply(lambda x: x/total_btf , axis=1)
    
    matrix_ntf2['ntf2'] = matrix_ntf2.apply(sum, axis=1)

    matrix['ntf2'] = matrix_ntf2['ntf2']

    return matrix[['word','totalFrequency','ntf1','ntf2']]

In [36]:
ntf_matrix = get_ntf(tf_matrix)

In [37]:
def get_ntf_idf(ntf, df):

    ntf = ntf.set_index('word')
    ntf_idf = pd.concat([ntf, df], axis=1)


    def get_btfidf(scores):
        global document_count
        return (np.log(scores['totalFrequency']) + 1.0) * np.log(document_count/scores['df'])

    def get_ntf1idf(scores):
        global document_count
        return (np.log(scores['ntf1']) + 1.0) * np.log(document_count/scores['df'])

    def get_ntf2idf(scores):
        global document_count
        return (np.log(scores['ntf2']) + 1.0) * np.log(document_count/scores['df'])

    
    ntf_idf['btf_idf'] = ntf_idf.apply(get_btfidf, axis=1)
    ntf_idf['ntf1_idf'] = ntf_idf.apply(get_ntf1idf, axis=1)
    ntf_idf['ntf2_idf'] = ntf_idf.apply(get_ntf2idf, axis=1)

    btf_rank = ntf_idf['btf_idf'].sort_values(ascending=False)
    ntf1_rank = ntf_idf['ntf1_idf'].sort_values(ascending=False)
    ntf2_rank = ntf_idf['ntf2_idf'].sort_values(ascending=False)

    return btf_rank, ntf1_rank, ntf2_rank

In [38]:
btf_rank , ntf1_rank, ntf2_rank = get_ntf_idf(ntf_matrix, df_matrix)

In [39]:
btf_rank.to_csv('../../Data/btf.csv', encoding='utf-8-sig')

In [40]:
ntf1_rank.to_csv('../../Data/ntf1.csv',encoding='utf-8-sig')

In [41]:
ntf2_rank.to_csv('../../Data/ntf2.csv', encoding='utf-8-sig')